In [1]:
! hostname
! whoami
! pwd
# ! nvidia-smi

dhcp-10-29-107-41.dyn.MIT.EDU
akarshkumar0101
/Users/akarshkumar0101/exploration-generalization/atari


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%run imports.py

In [5]:
[k for k in gym.registry.keys() if 'montezuma' in k.lower()]

['MontezumaRevenge-v0',
 'MontezumaRevengeDeterministic-v0',
 'MontezumaRevengeNoFrameskip-v0',
 'MontezumaRevenge-v4',
 'MontezumaRevengeDeterministic-v4',
 'MontezumaRevengeNoFrameskip-v4',
 'MontezumaRevenge-ram-v0',
 'MontezumaRevenge-ramDeterministic-v0',
 'MontezumaRevenge-ramNoFrameskip-v0',
 'MontezumaRevenge-ram-v4',
 'MontezumaRevenge-ramDeterministic-v4',
 'MontezumaRevenge-ramNoFrameskip-v4',
 'ALE/MontezumaRevenge-v5',
 'ALE/MontezumaRevenge-ram-v5']

In [148]:
env.envs[0].env.env.env.get_action_meanings()

['NOOP',
 'FIRE',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT',
 'UPFIRE',
 'RIGHTFIRE',
 'LEFTFIRE',
 'DOWNFIRE',
 'UPRIGHTFIRE',
 'UPLEFTFIRE',
 'DOWNRIGHTFIRE',
 'DOWNLEFTFIRE']

In [4]:
env = make_env('BreakoutNoFrameskip-v4', n_envs=8, device='mps')

NameNotFound: Environment `BreakoutNoFrameskip-v4` doesn't exist in namespace ALE. Did you mean: `Breakout-ram`?

In [325]:
env.single_action_space, env.action_space, env.single_observation_space, env.observation_space

(Discrete(18),
 MultiDiscrete([18 18 18 18 18 18 18 18]),
 Box(0, 255, (4, 84, 84), uint8),
 Box(0, 255, (8, 4, 84, 84), uint8))

In [326]:

obs, info = env.reset()
for _ in tqdm(range(2048)):
    obs, rew, term, trunc, info = env.step(env.action_space.sample())
    # if rew.max()>0:
    #     print(rew.max())

  0%|          | 0/2048 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [252]:
ret_ext = torch.cat(env.key2past_rets['ret_ext']).tolist()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0]

In [277]:
10000000//(128*8)

9765

In [288]:
9765//100

97

In [289]:
97*10

970

In [290]:
970/97

10.0

In [ ]:
torch.from_numpy(rew)

tensor([0., 0., 0., 0., 0., 0., 0., 0.])

In [298]:
x = info['obs']
x.shape, x.dtype, x.device

(torch.Size([8, 84, 84]), torch.uint8, device(type='mps', index=0))

In [32]:
import random

In [33]:
random.seed(0)
np.random.seed(0)
env = make_env('Breakout', n_envs=8, device='mps', seed=0)
obs, info = env.reset()
for _ in tqdm(range(128)):
    obs, rew, term, trunc, info = env.step(env.action_space.sample())
info['obs'].sum().item()

  0%|          | 0/128 [00:00<?, ?it/s]

9408985

In [34]:
random.seed(0)
np.random.seed(0)
env = make_env('Breakout', n_envs=8, device='mps', seed=0)
obs, info = env.reset()
for _ in tqdm(range(128)):
    obs, rew, term, trunc, info = env.step(env.action_space.sample())
info['obs'].sum().item()

  0%|          | 0/128 [00:00<?, ?it/s]

9408985

In [41]:
from agent_atari import *
import torchinfo

In [42]:

# torchinfo.summary()

In [45]:
agent = Agent(env)

torchinfo.summary(agent, input_size=(args.batch_size, 4, 84, 84))

Layer (type:depth-idx)                   Output Shape              Param #
Agent                                    [256]                     --
├─Sequential: 1-1                        [256, 512]                --
│    └─Conv2d: 2-1                       [256, 32, 20, 20]         8,224
│    └─ReLU: 2-2                         [256, 32, 20, 20]         --
│    └─Conv2d: 2-3                       [256, 64, 9, 9]           32,832
│    └─ReLU: 2-4                         [256, 64, 9, 9]           --
│    └─Conv2d: 2-5                       [256, 64, 7, 7]           36,928
│    └─ReLU: 2-6                         [256, 64, 7, 7]           --
│    └─Flatten: 2-7                      [256, 3136]               --
│    └─Linear: 2-8                       [256, 512]                1,606,144
│    └─ReLU: 2-9                         [256, 512]                --
├─Linear: 1-2                            [256, 18]                 9,234
├─Linear: 1-3                            [256, 1]               

In [50]:
(args.batch_size,) + env.single_observation_space.shape

(256, 4, 84, 84)